In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/My Drive/Master Thesis 2020

In [ ]:
# !ls

In [ ]:
import pandas as pd
import os
import numpy as np
import xlrd
import copy
import datetime
import glob
import time
# from __future__ import division, print_function, unicode_literals

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns; sns.set()
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt
# plt.rc("font", size=14)
# mpl.rc('axes', labelsize=14)
# mpl.rc('xtick', labelsize=12)
# mpl.rc('ytick', labelsize=12)
%matplotlib inline

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
# Reference
# https://nbviewer.jupyter.org/github/ageron/handson-ml/blob/master/04_training_linear_models.ipynb

### Import data


In [ ]:
df_uk = pd.read_csv('elexon_all_uk.csv', sep=',', parse_dates=True, index_col=[0], engine='python') # TO-DO: make sure the timeblokcs are continuous and with 30 min timeblocks
df_uk = df_uk.rename(columns={'Unnamed: 0': 'timeblock'})
df_uk.index.dtype # '<M8[ns]'is time series

In [ ]:
df_uk.wind_elexon = pd.to_numeric(df_uk.wind_elexon, errors='coerce') # wind_elexon contains obejcts


In [ ]:
df_uk.shape

### Add columns

In [ ]:
# Create new column y: intraday price - imbalance price >= 0  then denote as 1; intraday price - imbalance price <0 then denote as 0
# TO-DO: ROC curve https://www.google.com/url?sa=i&url=https%3A%2F%2Fmachinelearningmastery.com%2Froc-curves-and-precision-recall-curves-for-classification-in-python%2F&psig=AOvVaw0v5zXIpH4HfiuTZ-tlMgrC&ust=1595149731542000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCOCF0e-51uoCFQAAAAAdAAAAABAD
# TO-DO: show how confident we should
def create_y_column(row): # Alternative: where method
    if row['intraday_price'] < row['imbalance_price']: # Learning: Use lambda function
        val = 0
    else:
        val = 1

    return val

# Add y 
df_uk['y'] = df_uk.apply(create_y_column, axis=1)

In [ ]:
# Add error columns
df_uk_error = df_uk.copy()

df_uk_error['wind_entsoe_error'] = df_uk_error['wind_for_entsoe'] - df_uk_error['wind_entsoe']
df_uk_error['wind_elexon_error'] = df_uk_error['wind_for_elexon'] - df_uk_error['wind_elexon']
df_uk_error['solar_entsoe_error'] = df_uk_error['solar_for_entsoe'] - df_uk_error['solar_entsoe']
# df_uk_error.head()

In [ ]:
df_uk = df_uk_error

In [ ]:
# Add weekday/weekend dummies
df_uk['weekday'] = ((pd.DatetimeIndex(df_uk.index).dayofweek) // 5 == 1).astype(float)

In [ ]:
# Add consin sin
df_uk['timestamps'] = df_uk.index
dateofyr = df_uk.timestamps.dt.dayofyear
df_uk['timestamps_trigonometric'] = np.sin(dateofyr*2*np.pi/365)
df_uk = df_uk.drop(columns='timestamps')
# df_uk

In [ ]:
# df_uk.head(72)

In [ ]:
df_uk.columns

### Modeling - Random Forest



##### Data prep (Full variables)

In [ ]:
df_uk_rf = df_uk.copy()
df_uk_rf.head()

In [ ]:
# Set hyperparameters
# Create time lag columns
l = 672 # l is lag

# Random Forest classifier parameters
n_estimators = 500
max_features = None

In [ ]:
# Create t-2, t-3... columns

###
p = 6 # for lag 0.5 hr: 2 # for lag 2 hr: 6

lags = range(p, p+l)  # 5 lags # 10 lags # 48 lags

df_uk_rf = df_uk_rf.assign(**{ # Jeffrey: How does this work?
    '{} (t-{})'.format(col, t): df_uk_rf[col].shift(t)
    for t in lags
    for col in df_uk_rf[['demand_INDO','demand_ITSDO']]
})

###
p = 8  # for lag 0.5 hr: 4 # for lag 2 hr: 8

lags = range(p, p+l) 

df_uk_rf = df_uk_rf.assign(**{ # Jeffrey: How does this work?
    '{} (t-{})'.format(col, t): df_uk_rf[col].shift(t)
    for t in lags
    for col in df_uk_rf[['wind_entsoe', 'solar_entsoe', 'wind_elexon', 'wind_entsoe_error','solar_entsoe_error', 'wind_elexon_error']]
})

###
p = 0 

lags = range(p, p+l) 

df_uk_rf = df_uk_rf.assign(**{ # Jeffrey: How does this work?
    '{} (t-{})'.format(col, t): df_uk_rf[col].shift(t)
    for t in lags
    for col in df_uk_rf[['wind_for_entsoe','solar_for_entsoe', 'wind_for_elexon', 'timestamps_trigonometric']]
})


###
p = 7  # for lag 0.5 hr: 3 # for lag 2 hr: 7

lags = range(p, p+l) 

df_uk_rf = df_uk_rf.assign(**{ # Jeffrey: How does this work?
    '{} (t-{})'.format(col, t): df_uk_rf[col].shift(t)
    for t in lags
    for col in df_uk_rf[['imbalance_price','imbalance_volume', 'intraday_volume', 'intraday_price']]
})
df_uk_rf.head()

In [ ]:
df_uk_rf = df_uk_rf.reindex(sorted(df_uk_rf.columns), axis=1) # No need to standardize the dataset for RF - Standardization gives the same result in RF
df_uk_rf.head()

In [ ]:
# Check NA and extract the last value of the na's df
df_na = df_uk_rf[df_uk_rf.isna().any(axis=1)].iloc[[-1]].index.to_frame()
na_end = df_na.iloc[0,0]
na_end

In [ ]:
# Start the df with the first row with no NAs
df_uk_rf = df_uk_rf.loc[na_end + datetime.timedelta(minutes=30): , :]
df_uk_rf = df_uk_rf.drop(columns={'wind_for_entsoe','solar_for_entsoe', 'wind_for_elexon', 'timestamps_trigonometric',
                                  'demand_INDO','demand_ITSDO', 
                                  'wind_entsoe','solar_entsoe', 'wind_elexon',
                                  'wind_entsoe_error','solar_entsoe_error','wind_elexon_error',
                                  'imbalance_price','imbalance_volume',
                                  'intraday_volume', 'intraday_price',})
df_uk_rf.head()

In [ ]:
df_uk_rf[df_uk_rf.isnull().any(axis = 1)]

In [ ]:
# Split the dataset into training, validationand testing set
df_train = df_uk_rf['2017':'2018']
df_vali = df_uk_rf['2019-01':]
# df_test = df_uk_rf['2019-09':]

In [ ]:
# df_train[df_train.isnull().any(axis = 1)]

In [ ]:
# df_vali[df_vali.isnull().any(axis = 1)]

In [ ]:
# df_test[df_test.isnull().any(axis = 1)]

In [ ]:
# Shuffle the rows # Need to shuffle before dividing x and y, otherwise they will not match
df_train_shf = df_train.sample(frac=1) 
df_vali_shf = df_vali.sample(frac=1) 

In [ ]:
train = df_train_shf.loc[:, df_train_shf.columns != 'y']
train_labels = np.array(df_train_shf.pop('y')) 

vali = df_vali_shf.loc[:, df_vali_shf.columns != 'y']
vali_labels = np.array(df_vali_shf.pop('y')) 

In [ ]:
features = list(train.columns)

In [ ]:
# Validation
from sklearn.ensemble import RandomForestClassifier
t0 = time.time()

# Create the model
model = RandomForestClassifier(n_estimators = n_estimators,  # this is # of trees
                               bootstrap = True, 
                               max_features = max_features, random_state = 0) # levels:  5 to 7 -> 2^5, 2^7 = 128 # TO-DO: Where is the tree-pruning parameter? -> Try & error
# Fit on training data
model.fit(train, train_labels)

t1 = time.time()

total = t1-t0
print(model)
print('Run time(s)', str(datetime.timedelta(seconds=total)))

In [ ]:
n_nodes = []
max_depths = []

# Stats about the trees in random forest
for ind_tree in model.estimators_:
    n_nodes.append(ind_tree.tree_.node_count) # try: 50 or 100 of trees
    max_depths.append(ind_tree.tree_.max_depth) #  too deep is over-fitting 5-7 levels are already good
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
# Plot importance method 1
feat_importances = pd.Series(model.feature_importances_, index=train.columns)
feat_importances.nlargest(15).plot(kind='barh', color = 'royalblue')
plt.tight_layout()
filename = 'feature_importance_lag{}_ntrees{}_maxlevel{}_test'.format(l, n_estimators , max_features) # 
plt.savefig(filename+'.png')
# Plot importance method 1# More colorss check here https://matplotlib.org/2.0.0/examples/color/named_colors.html

In [ ]:
# Plot importance method 2 # Better -  easy to modify the plot # HELPPP: Wrong ytickes 
#https://stackoverflow.com/questions/332289/how-do-you-change-the-size-of-figures-drawn-with-matplotlib
feat_importances = pd.Series(model.feature_importances_, index=train.columns)
importances = feat_importances.nlargest(4)
# importances = model.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
# plt.show()

In [ ]:
# Training predictions (to demonstrate overfitting)
train_rf_predictions = model.predict(train)
train_rf_probs = model.predict_proba(train)[:, 1]

# Testing predictions (to determine performance)
rf_predictions = model.predict(vali)
rf_probs = model.predict_proba(vali)[:, 1]

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Plot formatting
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 18

def evaluate_model(predictions, probs, train_predictions, train_probs):
    """Compare machine learning model to baseline performance.
    Computes statistics and shows ROC curve."""
    
    baseline = {}
    
    baseline['recall'] = recall_score(vali_labels, 
                                     [1 for _ in range(len(vali_labels))])
    baseline['precision'] = precision_score(vali_labels, 
                                      [1 for _ in range(len(vali_labels))])
    baseline['roc'] = 0.5
    
    results = {}
    
    results['recall'] = recall_score(vali_labels, predictions)
    results['precision'] = precision_score(vali_labels, predictions)
    results['roc'] = roc_auc_score(vali_labels, probs)
    
    train_results = {}
    train_results['recall'] = recall_score(train_labels, train_predictions)
    train_results['precision'] = precision_score(train_labels, train_predictions)
    train_results['roc'] = roc_auc_score(train_labels, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(f'{metric.capitalize()} Baseline: {round(baseline[metric], 2)} Test: {round(results[metric], 2)} Train: {round(train_results[metric], 2)}')
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(vali_labels, [1 for _ in range(len(vali_labels))])
    model_fpr, model_tpr, _ = roc_curve(vali_labels, probs)

    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 14
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); 
    plt.ylabel('True Positive Rate'); plt.title('ROC Curves');
    plt.show()

evaluate_model(rf_predictions, rf_probs, train_rf_predictions, train_rf_probs)
filename = 'roccurve_lag{}_ntrees{}_maxlevel{}'.format(l, n_estimators , max_features) # Jeffrey: can't print
plt.savefig(filename+'.png')

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Oranges):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    # Plot the confusion matrix
    plt.figure(figsize = (10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 22)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize = 14,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True Label', size = 14)
    plt.xlabel('Predicted Label', size = 14)

# Confusion matrix
cm = confusion_matrix(vali_labels, rf_predictions)
plot_confusion_matrix(cm, classes = ['ITP < SSP', 'ITP >= SSP'],
                      title = 'Confusion Matrix')
plt.tight_layout()
filename = 'confmatrix_lag{}_ntrees{}_maxlevel{}'.format(l, n_estimators , max_features)
plt.savefig(filename+'.png')

In [ ]:
## Confusion Matrix

from sklearn.metrics import confusion_matrix
print('Confusion matrix', confusion_matrix(vali_labels, rf_predictions))

# Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy score',accuracy_score(vali_labels, rf_predictions))

# Recall
from sklearn.metrics import recall_score
print('Recall score', recall_score(vali_labels, rf_predictions, average=None))

# Precision
from sklearn.metrics import precision_score
print('Precision score',precision_score(vali_labels, rf_predictions, average=None))

# F1 score
from sklearn.metrics import f1_score
print('F1 score', f1_score(vali_labels, rf_predictions))
# The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. 
# The formula for the F1 score is: F1 = 2 * (precision * recall) / (precision + recall)